# Genius song lyrics : Data vizualisation

In [1]:
import sys

# enable modules import
sys.path.insert(1, '/kaggle/input/modules')

In [2]:
# install dependencies
!pip install pyspark
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=6a1b9e1af53ff4f582d432e7ecc6c002b1fb127d65aacb34ed706f5c8ae78f98
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 73.1 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=818403 sha256=e2b1eda6ba81ad07d3aa6ef532ceca7c4aba6e28ce26e8ebe69

In [3]:
from spark_preprocess import SparkSPreprocessor

sp = SparkSPreprocessor(
    input_path="/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv",
    parquet_path="/kaggle/working/data/",
    parquet_name = "song_lyrics.parquet",
    driver_memory = "20g"
)

514it [05:09,  1.66it/s]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 07:09:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = sp.preprocess_data(freq=0.5, seed = 42, sample_by='ddecade')

In [5]:
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,decade,ddecade
0,Its Hot Some Like It Hot,rap,JAY-Z,1999,103549,{},"[Produced by Timbaland]\n\n[Verse 1]\nYo, show...",18,en,en,en,1990,1980
1,Its Like That,rap,JAY-Z,1998,37692,"{""Kid Capri""}","[Intro: Jay-Z, Kid Capri & Both]\nYeah, uh-huh...",123,en,en,en,1990,1980
2,Hey AZ,rap,AZ,1997,7066,{SWV},"[Intro: AZ]\nWhat's the deal, son?\nIt's the w...",8718,en,en,en,1990,1980
3,Me My Bitch,rap,The Notorious B.I.G.,1994,495985,{Diddy},"[Intro: Notorious B.I.G., Puff Daddy & Girl]\n...",46,en,en,en,1990,1980
4,Back That Azz Up,rap,Juvenile,1998,506949,"{""Lil Wayne"",""Mannie Fresh""}",[Intro: Juvenile]\nCash Money Records taking o...,57,en,en,en,1990,1980


In [6]:
df.dtypes

title            object
tag              object
artist           object
year              int64
views             int64
features         object
lyrics           object
id                int64
language_cld3    object
language_ft      object
language         object
decade            int32
ddecade           int32
dtype: object

In [7]:
from vizualisation import barplot_by_decades

barplot_by_decades(df, group_by = 'ddecade')

In [9]:
from vizualisation import piechart_tags_decades

piechart_tags_decades(df, rows = 1, cols = 3, group_by = 'ddecade')

In [10]:
from vizualisation import top_artists_decades


top_artists_decades(df, decades=[1960, 1980, 2000], rows = 2, cols = 2, n_artists = 7, group_by='ddecade')

In [11]:
from preprocess_text import clean_lyrics

df = clean_lyrics(df)
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,decade,ddecade
0,Its Hot Some Like It Hot,rap,JAY-Z,1999,103549,{},yo show closer j-to-the-a-y-hovah place s...,18,en,en,en,1990,1980
1,Its Like That,rap,JAY-Z,1998,37692,"{""Kid Capri""}",yeah uh-huh watch this y'all uhh watch this ...,123,en,en,en,1990,1980
2,Hey AZ,rap,AZ,1997,7066,{SWV},what's the deal son it's the world famous az...,8718,en,en,en,1990,1980
3,Me My Bitch,rap,The Notorious B.I.G.,1994,495985,{Diddy},yo let— let— let me ask you a question yo yo...,46,en,en,en,1990,1980
4,Back That Azz Up,rap,Juvenile,1998,506949,"{""Lil Wayne"",""Mannie Fresh""}",cash money records taking over for the ' & ...,57,en,en,en,1990,1980


In [12]:
import spacy

# small model /!\ take the bigger one for Kaggle
new_nlp = spacy.load('en_core_web_sm')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning:

unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning:

file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tensorflow13GcsFileSystemE']



Could take some times ... (depend on the sample data size)

In [ ]:
from preprocess_text import naive_preprocess

df['tok_lyrics'] = df['lyrics'].apply(lambda text : naive_preprocess(text, new_nlp))

In [ ]:
df['unique_words'] = df['tok_lyrics'].apply(lambda x: len(set(x)))

In [ ]:
tags = df['tag'].unique().tolist()
tags.sort()

In [ ]:
words_decades(df, decades = [1960, 1980, 2000], group_by = 'ddecade')